In [665]:
import random

#### Parámetros

In [666]:
# Ejemplo del enunciado
T = 24*60 # 24h en minutos
# T = 100 #un mes en minutos #pongo 1 dia

nPistas = 3
nCoches = 20
nPuertas = 50

# lambda0 = getlambda0(t)
mu11 = 10.007615178514001
sigma11 = 3.0377002745229005
a12 = 10
b12 = 15
lambda2 = 1 / 2
lambda3 = 0.1

# T, nCoches, nPuertas, mu11, sigma11, a12, b12, lambda2, lambda3

In [667]:
def getlambda0(t):
    # t está en minutos, convertir a horas:
    t = t/60
    t = t%24 ##########ojo esto##########
    # Intervalo de 0 a 5 horas, lineal de 5 a 7
    if 0 <= t < 5:
        return 5 + (7 - 5) * (t - 0) / (5 - 0)
    # Intervalo de 5 a 8 horas, lineal de 7 a 6
    elif 5 <= t < 8:
        return 7 + (6 - 7) * (t - 5) / (8 - 5)
    # Intervalo de 8 a 15 horas, lineal de 6 a 9
    elif 8 <= t < 15:
        return 6 + (9 - 6) * (t - 8) / (15 - 8)
    # Intervalo de 15 a 17 horas, lineal de 9 a 6
    elif 15 <= t < 17:
        return 9 + (6 - 9) * (t - 15) / (17 - 15)
    # Intervalo de 17 a 24 horas, lineal de 6 a 5
    elif 14 <= t <= 24:
        return 6 + (5 - 6) * (t - 17) / (24 - 17)
    # Si el tiempo está fuera del rango de 0 a 24 horas
    else:
        return 10E9  

#### Funciones auxiliares para generación de tiempos

In [668]:
# Funciones auxiliares generacion de tiempos
def t_llegada(t):
    return random.expovariate(getlambda0(t))*60 #nº muy pequeños, tiene sentido?

def t_aterrizaje():
    # return np.random.normal(mu11, sigma11)
    # return random.normalvariate(mu11, sigma11) 
    while True:  # Bucle infinito hasta obtener un número válido
        resultado = random.normalvariate(mu11, sigma11)
        if resultado > 0:  # Comprobar si el resultado es mayor que cero
            return resultado
        
def t_traslado(): #N1- N2
    return random.expovariate(1/lambda2)

def t_desembarques():
    return random.expovariate(1/lambda3)

def t_salidas():
    return random.uniform(a12, b12)

#### Variables

In [669]:
t = 0                   # Variable reloj. tiempo asíncrono.
n1 = n2 = n3 = 0        # Variables de estado nº clientes
NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
NS1 = NS2 = NS3 = 0     # Variables contador nº salidas

# Lista de tiempos de eventos
LISTA = {
    'tLL1': 10E9,   
    'tS1':  [10E9]*nPistas,
    'tS2':  [10E9]*nCoches,
    'tS3':  [10E9]*nPuertas,
    'tSS':  [10E9]*nPistas
}

# Arrays for arrival and departure times
LL1 = []
LL2 = []
LL3 = []
S1 = []
S2 = []
S3 = []

# Colas
esperando_a_despegar = 0
esperando_a_aterrizar = 0
esperando_coche = 0
esperando_puerta = 0

n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas

#### Rutina de aterrizaje

In [670]:

def rutina_aterrizaje(t_suc):
    
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3
    global esperando_a_despegar, esperando_a_aterrizar, esperando_coche, esperando_puerta
    
    # para luego calcular numero medio de clientes
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    # Si tengo pista libre, aterrizo
    if (n1 < nPistas):
        # Actualizo variables contadores
        n1 += 1                     # El avión entra en n1, y ocupa una pista.
        NLL1 += 1                   # numero de llegadas al n1
        LL1.append(t_suc)
        t = t_suc
        
        Z = t_aterrizaje() #random.normalvariate(mu11, sigma11) #genero tiempo aterrizaje
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Z
        
        print(f'----ATERRIZAJE: {n1} pistas ocupadas. Minuto: {t}. Fin de maniobra: {t+Z}')
    
    # si no tengo pista, no aterrizo, me pongo en cola aterrizaje
    else:
        esperando_a_aterrizar+=1
        print(f'----ESPERANDO ATERRIZAR: {esperando_a_aterrizar} aviones esperando. Minuto {t}')

    #=========================================================================#

    # Genero tiempo siguente llegada
    Y = t_llegada(t)
    # Compruebo si el siguiente avión llega antes de T
    if (t + Y < T):
        LISTA['tLL1'] = t + Y #añado otro tiempo de llegada
        # print(f'----NUEVO AVIÓN: Minuto: {t+Y}')
    else:
        print(f'#### HORIZONTE ALCANZADO: {t+Y} > {T}, no llegan más aviones ####')
        print(f'#### n1 = {n1}, n2 = {n2}, n3 = {n3} ####')
        print(LISTA)


#### Rutina de traslado

In [671]:
def rutina_traslado(t_suc): 
    # un coche recoge al avión y lo desaloja de la pista (salida de N1)
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3
    global esperando_a_despegar, esperando_a_aterrizar, esperando_coche, esperando_puerta
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    t=t_suc     # Actualizamos tº de simulación
    
    n1 -=1      # El avión sale del nodo 1, y despeja una pista
    
    # si hay coches disponibles -> traslado
    if n2 < nCoches:
        
        n2 +=1                  # El avion entra en n2, y ocupa un coche
        
        # tiempo de salida del N2, que se corresponde con el final del traslado
        Z = t_traslado()
        LISTA['tS2'][LISTA['tS2'].index(10E9)] = t + Z 

        print(f'-----TRASLADO: {n2} coches ocupados. Minuto {t}. Fin de traslado {t+Z}')
        
    else: #si no hay coche disponibles 
        esperando_coche +=1 #lo pongo a la cola de ser trasladado
        print('-----ESPERANDO COCHE: {esperando_coche} aviones esperando. Minuto {t}')

    if esperando_a_aterrizar > 0: # ahora que se han liberado pistas, atendemos a la cola de aterrizaje
        Z = t_aterrizaje()                              # genero tiempo que tardará en aterrizar
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Z  # registro el tiempo en el que saldrá el avión del nodo 1
        
        n1+=1                       # El avión en cola entra al n1, ocupa una pista,
        esperando_a_aterrizar -= 1  # y libera su puesto en la cola.

        print(f'----ATERRIZAJE_cola: {n1} pistas ocupadas. Minuto: {t}. Fin de maniobra: {t+Z}')
        
    elif esperando_a_despegar > 0:

        if esperando_a_aterrizar>0: return -1
        
        n1+=1                       # El avión en cola entra al n1, ocupa una pista,
        esperando_a_despegar -=1    # Y libera su puesto en la cola.

        Z = t_salidas()                                 # genero tiempo que tardará en despegar
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + Z  # registro el tiempo en el que saldrá el avión del nodo 1

        
        print(f'-------DESPEGUE_cola2: {n1} pistas ocupadas. Minuto: {t}. Fin de maniobra: {t+Z}')
        

#### Rutina de desembarque

In [672]:
def rutina_desembarque(t_suc): 
    # avion en aeropuerto: desembarcan pasajeros, embarcan los nuevos y traslado a pistas
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3
    global esperando_a_despegar, esperando_a_aterrizar, esperando_coche, esperando_puerta
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    t=t_suc     # Actualizamos tº de simulación
    
    n2 -=1      # El avión sale de n2 y libera su vehículo.
    
    #si hay puertas de embarque disponibles
    if n2 < nPuertas:

        n3 +=1      # El avión entra en n3, y ocupa una puerta
        
        Z = t_desembarques()
        LISTA['tS3'][LISTA['tS3'].index(10E9)] = t + Z

        print(f'------PUERTA ASIGNADA: {n3} puertas ocupadas. Minuto: {t}. Fin de proceso: {t+Z}')
        
    else: # si no hay puertas disponibles 
        esperando_puerta+=1     # esperando puerta embarque
        print(f'------ESPERANDO PUERTA: {esperando_puerta} aviones esperando. Minuto: {t}')

    # si tengo aviones pendientes de ser trasladados
    if esperando_coche > 0: 

        n2+=1                       # El avión en cola entra al n2, ocupa un coche,
        esperando_coche -= 1        # Y libera su puesto en la cola.
        
        Y = t_traslado()
        LISTA['tS2'][LISTA['tS2'].index(10E9)] = t + Y

        print(f'-----TRASLADO_cola: {n2} coches ocupados. Minuto {t}. Fin de traslado {t+Z}')

#### Rutina de despegue (vuelta al nodo 1)

In [673]:
def rutina_despegue(t_suc): 
    # sale del nodo 3 y espera a poder salir del sistema (nodo SS)    
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3
    global esperando_a_despegar, esperando_a_aterrizar, esperando_coche, esperando_puerta
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    
    t=t_suc     # Actualizamos tº de simulación

    n3 -=1      # El avión sale del n3 y libera un puerta.
    
    # Si hay pistas libres
    if n1 < nPistas:
        
        n1+=1               # El avión entra al n1 y ocupa una pista

        # generamos tiempo de despegue
        Z = t_salidas()
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + Z

        print(f'-------DESPEGUE: {n1} pistas ocupadas. Minuto: {t}. Fin de maniobra: {t+Z}')

    else:    
        esperando_a_despegar+=1 #esperando para el despegue
        print(f'-------ESPERANDO DESPEGAR: {esperando_a_despegar} aviones esperando. Minuto: {t}')

    # Manejo la cola del nodo anterior
    # si quedan aviones esperando puerta
    if esperando_puerta > 0:
        
        n3+=1                   # El avion entra al n3, ocupa una puerta
        esperando_puerta-=1     # Y libera su puesto en la cola

        Y = t_desembarques()    # Genero tiempo de desembarque
        LISTA['tS3'][LISTA['tS3'].index(10E9)] = t + Y

        print(f'------PUERTA ASIGNADA_cola: {n3} puertas ocupadas. Minuto: {t}. Fin de proceso: {t+Z}')

#### Rutina de salida

In [674]:
def rutina_salida_sistema(t_suc): 
    # sale del nodo 3 y espera a poder salir del sistema (nodo SS)    
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3
    global esperando_a_despegar, esperando_a_aterrizar, esperando_coche, esperando_puerta
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    t=t_suc     # Actualizamos tº de simulación

    n1 -=1      # El avión sale del nodo 1 y despeja una pista.
    print(f'--------SALIDA DE SISTEMA: {n1} pistas ocupadas. Minuto: {t}.')
    print(f'-------------------------: Esperando aterrizar: {esperando_a_aterrizar}, Esperando despegar: {esperando_a_despegar}')

    # Manejo la cola del nodo anterior  
    if esperando_a_aterrizar > 0: # ahora que se han liberado pistas, atendemos a la cola de aterrizaje
        Z = t_aterrizaje()                              # genero tiempo que tardará en aterrizar
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Z  # registro el tiempo en el que saldrá el avión del nodo 1
        
        n1+=1                       # El avión en cola entra al n1, ocupa una pista,
        esperando_a_aterrizar -= 1  # y libera su puesto en la cola.

        print(f'----ATERRIZAJE_cola: {n1} pistas ocupadas. Minuto: {t}. Fin de maniobra: {t+Z}')
        
    elif esperando_a_despegar > 0:

        if esperando_a_aterrizar>0: return -1
        
        n1+=1                       # El avión en cola entra al n1, ocupa una pista,
        esperando_a_despegar -=1    # Y libera su puesto en la cola.

        Z = t_salidas()                                 # genero tiempo que tardará en despegar
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + Z  # registro el tiempo en el que saldrá el avión del nodo 1

        
        print(f'-------DESPEGUE_cola4: {n1} pistas ocupadas. Minuto: {t}. Fin de maniobra: {t+Z}')

# MAIN

In [675]:

def simul_main():
    global LISTA

    # Genero primer evento (aterrizaje)
    X = t_llegada(0)
    
    # Caso especial
    if X > T:
        return
    
    rutina_aterrizaje(X)

    # Bucle principal
    while t < T+10E9 and any(min(val) != 10E9 if isinstance(val, list) else val != 10E9 for val in LISTA.values()):
        min_key = min(
            LISTA.keys(), 
            key=lambda k: min(LISTA[k]) if isinstance(LISTA[k], list) else LISTA[k]
        )
        print(LISTA)
        print(min_key, LISTA[min_key])
        print ()

        if(min_key=='tLL1'): # Si el próximo evento es un aterrizaje (LLN1)
            t_suc=LISTA['tLL1']
            LISTA['tLL1']= 10E9
            rutina_aterrizaje(t_suc)
            
            # print(LISTA)
            # print()
            
        elif(min_key=='tS1'): # Si el próximo evento es un traslado (SN1)
            t_suc=min(LISTA['tS1'])
            LISTA['tS1'][LISTA['tS1'].index(min(LISTA['tS1']))] = 10E9
            rutina_traslado(t_suc)
            
            # print(LISTA)
            # print()
            
        elif(min_key=='tS2'): # Si el próximo evento es un deembarque (SN2)
            t_suc=min(LISTA['tS2'])
            LISTA['tS2'][LISTA['tS2'].index(min(LISTA['tS2']))] = 10E9
            rutina_desembarque(t_suc)
            
            # print(LISTA)
            # print()
            
        elif(min_key=='tS3'): # Si el próximo evento es un deembarque (SN3)
            t_suc=min(LISTA['tS3'])
            LISTA['tS3'][LISTA['tS3'].index(min(LISTA['tS3']))] = 10E9
            rutina_despegue(t_suc)
            
            # print(LISTA)
            # print()
            
        elif(min_key=='tSS'): # Si el próximo evento es un deembarque (SN4)
            t_suc=min(LISTA['tSS'])
            LISTA['tSS'][LISTA['tSS'].index(min(LISTA['tSS']))] = 10E9
            
            rutina_salida_sistema(t_suc)
            
            # print(LISTA)
            # print()

    

In [676]:
simul_main()

----ATERRIZAJE: 1 pistas ocupadas. Minuto: 4.911346862495535. Fin de maniobra: 13.74605932360506
{'tLL1': 14.653158781616533, 'tS1': [13.74605932360506, 10000000000.0, 10000000000.0], 'tS2': [10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0], 'tS3': [10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 1000

In [677]:
print(n1)
print(n2)
print(n3)
print(n_med_n1)
print(n_med_n2)
print(n_med_n3)

0
0
0
5900.883590204259
104.31213287935947
22.124996232803444
